In [17]:
# Imports
import glob
import numpy as np
from nilearn import image, plotting
from nilearn.image import new_img_like
import os
import scipy.io

In [68]:
main_folder = 'C:\\Users\\davide\\Documents\\GitHub\\wp1_2_roast' #main folder with data
bck_img_map = image.load_img(os.path.join(main_folder, 'MNI152_T1_1mm_Brain.nii')) #Background for plotting purposes
#List of subjects
subjects = ['01','02','03','04','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','22','23','24']
#subjects = ['01','02']

dcm_data_day1_path = os.path.join(main_folder,'Day1_all_EPvalues.mat') #mat file with DCM results of day 1
dcm_data_day1 = scipy.io.loadmat(dcm_data_day1_path)
#NB this file contains results for all 22 subjects for all 3 polarities (anodal, cathodal and sham)
#And the order is anodal, cathodal and sham for each subject (so I'm only interested in matrices 0, 3, 6 ... etc)

In [70]:
#Loading ROIs
m1_path = os.path.join(main_folder,'rois','m1.nii')
th_path = os.path.join(main_folder,'rois','th.nii')
m1 = image.load_img(m1_path)
th = image.load_img(th_path)

In [71]:
#range(0,len(files_with_res['ALLEP'][0]),3) extracting anodal
#dcm_data_day1['ALLEP'][0][i].todense().reshape((4,4))

In [73]:
pos_dcm_res = 0 #Position of the DCM results for subject 1, day 1 (anodal)

for sbj in subjects:
    print('Analysing subject: ' + sbj)
    path = 'D:\\roast-chapter3\wp2a\sub-' + sbj
    print('Path: ' + path)
    
    #Loading ROAST field magnitude simulation
    emag_map = image.load_img(glob.glob(path+ '/swsub-*_T1_*_emag.nii')) 
    emag_data = emag_map.get_fdata() 
    
    #Loading binary map for the subject (c1 = grey matter, c2 = white matter)
    anat_map = image.load_img(glob.glob(path+ '/c1c2bin.nii')) #Loading c1/c2 masks
    anat_data = anat_map.get_fdata()
    
    #Emag map masked using c1c2 masks (to keep only values in white/grey matter)
    emag_msk = np.where(anat_data > 0, emag_data, 0) 
    emag_map_msk = new_img_like(emag_map,emag_msk)
    
    # Resampling ROIs to the dimention of the emag map
    m1_resampled = image.resample_to_img(m1,emag_map_msk, interpolation = 'nearest')
    th_resampled = image.resample_to_img(th,emag_map_msk, interpolation = 'nearest')
    
    #Loading DCM results for the subject
    this_dcm = dcm_data_day1['ALLEP'][0][pos_dcm_res].todense().reshape((4,4))
    #print(this_dcm)
    
    pos_dcm_res += 3
    

Analysing subject: 01
Path: D:\roast-chapter3\wp2a\sub-01
Analysing subject: 02
Path: D:\roast-chapter3\wp2a\sub-02
Analysing subject: 03
Path: D:\roast-chapter3\wp2a\sub-03
Analysing subject: 04
Path: D:\roast-chapter3\wp2a\sub-04
Analysing subject: 06
Path: D:\roast-chapter3\wp2a\sub-06
Analysing subject: 07
Path: D:\roast-chapter3\wp2a\sub-07
Analysing subject: 08
Path: D:\roast-chapter3\wp2a\sub-08
Analysing subject: 09
Path: D:\roast-chapter3\wp2a\sub-09
Analysing subject: 10
Path: D:\roast-chapter3\wp2a\sub-10
Analysing subject: 11
Path: D:\roast-chapter3\wp2a\sub-11
Analysing subject: 12
Path: D:\roast-chapter3\wp2a\sub-12
Analysing subject: 13
Path: D:\roast-chapter3\wp2a\sub-13
Analysing subject: 14
Path: D:\roast-chapter3\wp2a\sub-14
Analysing subject: 15
Path: D:\roast-chapter3\wp2a\sub-15
Analysing subject: 16
Path: D:\roast-chapter3\wp2a\sub-16
Analysing subject: 17
Path: D:\roast-chapter3\wp2a\sub-17
Analysing subject: 18
Path: D:\roast-chapter3\wp2a\sub-18
Analysing subj

(157, 189, 156)

In [ ]:
plotting.plot_roi(mesh_map,
                  cmap = 'hot_r',
                  threshold = 0, vmax = 0.36, alpha = 0.45,
                  black_bg = False)

#image.math_img("img", img=mesh_map).to_filename('meshed.nii')


import matplotlib.pyplot as plt
hist, bins = np.histogram(mesh, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()